<a href="https://colab.research.google.com/github/naexuis/csScheduler/blob/main/mdh_python_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/naexuis/csScheduler.git

Cloning into 'csScheduler'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 28 (delta 8), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (28/28), 9.36 KiB | 1.04 MiB/s, done.


In [2]:
dir_path = "/content/csScheduler/"

In [3]:
import pandas as pd
import numpy as np
import os
import copy
import json
import matplotlib.pyplot as plt

In [4]:
try:
    import pyomo
except:
    %pip install -q Pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 5.3 MB/s eta 0:00:00


In [5]:
# Import Pyomo Library
import pyomo
from pyomo.environ import *

print(pyomo.__version__)

6.4.4


In [6]:
# Install COIN-OR IPOPT Solver
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [121]:
# Install COIN-OR Bonmin Solver
!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
!unzip -o -q bonmin-linux64

## Problem Description

A new store has been opened which will be open 16 hours a day, 5 days a week. 

Each day, there are **two eight-hour** shifts. 
* Morning shift is from 06:00 to 14:00
* Evening shift is from 14:00 to 22:00

In theory, during the day there are two workers, but this can change base on demand. 

Each worker will not exceed a maximum of 40 hours per week.

In principle, there are available ten employees.

In [7]:
# Define days (1 week)
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Enter shifts of each day
shifts = ['morning', 'evening']  # 2 shifts of 8 hours

In [8]:
# Opening JSON Schedule file to create nested dictionary
with open(dir_path+'Schedule.json') as json_file:
    input_schedule = json.load(json_file)

In [9]:
with open(dir_path+'People.json') as json_file:
    worker_schedule = json.load(json_file)

In [10]:
input_schedule

{'Monday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '2',
    'managerNeeded': 'false'},
   {'shifts': 'evening',
    'numberOfEmployeesNeeded': '2',
    'managerNeeded': 'false'}]},
 'Tuesday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '3',
    'managerNeeded': 'false'},
   {'shifts': 'evening',
    'numberOfEmployeesNeeded': '0',
    'managerNeeded': 'false'}]},
 'Wednesday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '2',
    'managerNeeded': 'false'},
   {'shifts': 'evening',
    'numberOfEmployeesNeeded': '1',
    'managerNeeded': 'false'}]},
 'Thursday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '2',
    'managerNeeded': 'false'},
   {'shifts': 'evening',
    'numberOfEmployeesNeeded': '1',
    'managerNeeded': 'false'}]},
 'Friday': {'timesWeNeedWorkers': [{'shifts': 'morning',
    'numberOfEmployeesNeeded': '0',
    'managerNeeded': 'false'}

In [11]:
len(worker_schedule['People'])

10

In [12]:
days_shifts = {day: shifts for day in days}
days_shifts

{'Monday': ['morning', 'evening'],
 'Tuesday': ['morning', 'evening'],
 'Wednesday': ['morning', 'evening'],
 'Thursday': ['morning', 'evening'],
 'Friday': ['morning', 'evening']}

In [84]:
work_shifts = {}

for k in range(len(days)):
    tmpLST = input_schedule[days[k]]['timesWeNeedWorkers']

    for i in range(len(tmpLST)):
        work_shifts[(days[k], tmpLST[i]['shifts'])] = int(tmpLST[i]['numberOfEmployeesNeeded'])

work_shifts

{('Monday', 'morning'): 2,
 ('Monday', 'evening'): 2,
 ('Tuesday', 'morning'): 3,
 ('Tuesday', 'evening'): 0,
 ('Wednesday', 'morning'): 2,
 ('Wednesday', 'evening'): 1,
 ('Thursday', 'morning'): 2,
 ('Thursday', 'evening'): 1,
 ('Friday', 'morning'): 0,
 ('Friday', 'evening'): 2}

In [14]:
# Enter workers ids (name, number, ...)
workers = ['W' + str(i) for i in range(1, 11)]  # 10 workers available, more than needed
workers

['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10']

In [15]:
# We need to create a dictionary of the workers availability where the 
# dictionary key will be a tuple and the value will be a binary variable
# ("worker", "Day", "Shift"): "Available"

c = {}
for k in range(len(worker_schedule['People'])):
    tmpLST = worker_schedule['People'][k]

    for i in range(len(days)):
        tmpLST2 = tmpLST[days[i]]['freeTimes']

        for j in range(len(tmpLST2)):
            c[(tmpLST['name'], days[i], tmpLST2[j]['shifts'])] = int(tmpLST2[j]['available'])

In [16]:
c

{('W1', 'Monday', 'morning'): 0,
 ('W1', 'Monday', 'evening'): 0,
 ('W1', 'Tuesday', 'morning'): 1,
 ('W1', 'Tuesday', 'evening'): 1,
 ('W1', 'Wednesday', 'morning'): 0,
 ('W1', 'Wednesday', 'evening'): 0,
 ('W1', 'Thursday', 'morning'): 0,
 ('W1', 'Thursday', 'evening'): 0,
 ('W1', 'Friday', 'morning'): 1,
 ('W1', 'Friday', 'evening'): 1,
 ('W2', 'Monday', 'morning'): 1,
 ('W2', 'Monday', 'evening'): 1,
 ('W2', 'Tuesday', 'morning'): 1,
 ('W2', 'Tuesday', 'evening'): 1,
 ('W2', 'Wednesday', 'morning'): 0,
 ('W2', 'Wednesday', 'evening'): 0,
 ('W2', 'Thursday', 'morning'): 1,
 ('W2', 'Thursday', 'evening'): 1,
 ('W2', 'Friday', 'morning'): 0,
 ('W2', 'Friday', 'evening'): 0,
 ('W3', 'Monday', 'morning'): 1,
 ('W3', 'Monday', 'evening'): 1,
 ('W3', 'Tuesday', 'morning'): 0,
 ('W3', 'Tuesday', 'evening'): 0,
 ('W3', 'Wednesday', 'morning'): 1,
 ('W3', 'Wednesday', 'evening'): 1,
 ('W3', 'Thursday', 'morning'): 1,
 ('W3', 'Thursday', 'evening'): 1,
 ('W3', 'Friday', 'morning'): 1,
 ('W3',

In [17]:
# We need to create a dictionary of the workers total desired hours

maxhr = {}
for k in range(len(worker_schedule['People'])):
    tmpLST = worker_schedule['People'][k]
    maxhr[tmpLST['name']] = int(tmpLST['hoursDesired'])

In [18]:
maxhr

{'W1': 40,
 'W2': 40,
 'W3': 40,
 'W4': 40,
 'W5': 40,
 'W6': 40,
 'W7': 40,
 'W8': 40,
 'W9': 40,
 'W10': 40}

Define $W$ as the set of workers, $S$ as the sets of shifts, and $H$ as the set of max number of hours.

Let $c_{ws}$ be worker $j$ availability for shift $s$

Let $x_{ws}$ be the proportion of shift $s$ that is completed by worker $j$.

$${minimize} \sum_{w \in W} \sum_{s \in S} c_{ws}x_{ws}$$

subject to the total numbers of hours $h$ for worker $j$

$$\sum_{s \in S}c_{ws}x_{ws} \le h_{j}$$

the sum of each shift that a worker completes must sum to unity.

$$\sum_{w \in W}x_{ws} = 1$$



In [122]:
M = ConcreteModel()

In [123]:
M.WORKERS = Set(initialize=workers)
M.DAYS = Set(initialize=days)
M.SHIFTS = Set(initialize=shifts)

In [124]:
# Parameters
M.c = Param(M.WORKERS, M.DAYS, M.SHIFTS, initialize=c, default=0)
M.max_hours = Param(M.WORKERS, initialize=maxhr)

In [125]:
M.b = Var(M.WORKERS, M.DAYS, M.SHIFTS, within=Binary, initialize=0)

In [126]:
M.obj = Objective(expr=quicksum(M.c[i, j, k] * M.b[i, j, k] for i in M.WORKERS for j in M.DAYS for k in M.SHIFTS), 
                  sense=minimize)

In [127]:
M.constraints = ConstraintList()  # Create a set of constraints

for i in M.WORKERS:
    M.constraints.add(
        M.max_hours[i] >= quicksum(8 * M.c[i, j, k] * M.b[i, j, k] for j in M.DAYS for k in M.SHIFTS)
    )

for j in M.DAYS:
    for k in M.SHIFTS:
        M.constraints.add(
            work_shifts[j, k] == quicksum(M.c[i, j, k] * M.b[i, j, k] for i in M.WORKERS)
        )

In [128]:
# M.pprint()

In [148]:
# solver1 = SolverFactory('ipopt', executable='/content/ipopt')
solver1 = SolverFactory('bonmin', executable='/content/bonmin')
solver1.solve(M)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 62, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'bonmin\\x3a Optimal', 'Termination condition': 'optimal', 'Id': 3, 'Error rc': 0, 'Time': 0.045145511627197266}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [154]:
dow = {}
for i in range(len(days)):
    dow[days[i]] = i

In [157]:
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(M.b, names=['w', 'd', 's']))
df['x'] = [value(M.b[key]) for key in df.index]
df['c'] = [M.c[key] for key in df.index]
df['res'] = [df['x'][i] * df['c'][i] for i in range(len(df))]
df = df.reset_index(drop=False)
df['dow'] = [dow[df['d'][i]] for i in range(len(df))]
df_res = df[df['res']==1].sort_values(by=['dow']).reset_index(drop=True)
df_res

,w,d,s,x,c,res,dow
0,W2,Monday,morning,1.0,1,1.0,0
1,W2,Monday,evening,1.0,1,1.0,0
2,W3,Monday,morning,1.0,1,1.0,0
3,W10,Monday,evening,1.0,1,1.0,0
4,W1,Tuesday,morning,1.0,1,1.0,1
5,W2,Tuesday,morning,1.0,1,1.0,1
6,W10,Tuesday,morning,1.0,1,1.0,1
7,W3,Wednesday,morning,1.0,1,1.0,2
8,W3,Wednesday,evening,1.0,1,1.0,2
9,W5,Wednesday,morning,1.0,1,1.0,2


In [159]:
df_gb = df_res.groupby(['w'])['res'].sum()
df_gb*8

w
W1     16.0
W10    32.0
W2     32.0
W3     32.0
W5      8.0
Name: res, dtype: float64